In [1]:
import sklearn
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import collections
from itertools import chain
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn import set_config
from sklearn.compose import make_column_selector, make_column_transformer

In [3]:
import warnings
warnings.filterwarnings('ignore')

# 1. Load Data & Merge to Single DataFrame

In [4]:
trades = pd.read_csv(Path("../data/raw_data/AAPL_trades.csv"))
quotes = pd.read_csv(Path("../data/raw_data/AAPL_quotes.csv"))

In [5]:
trades.head(10)

,Unnamed: 0,Time,Date,Exchange,Symbol,Trade_Volume,Trade_Price,Sale_Condition,Source_of_Trade,Trade_Stop_Stock_Indicator,Trade_Correction_Indicator,Sequence_Number,Trade_Id,Trade_Reporting_Facility,Participant_Timestamp,Trade_Reporting_Facility_TRF_Timestamp,Trade_Through_Exempt_Indicator
0,0,2020-01-02 04:00:00.064010,2020-01-02,P,AAPL,3801,295.05,@ T,N,NaN,0,1185,1,,40000063617792,NaN,1
1,1,2020-01-02 04:00:02.828485,2020-01-02,P,AAPL,1,295.08,@FTI,N,NaN,0,1195,2,,40002828108800,NaN,1
2,2,2020-01-02 04:00:06.250392,2020-01-02,Q,AAPL,6,295.25,@ TI,N,NaN,0,1197,1,,40006250366823,NaN,0
3,3,2020-01-02 04:00:06.429757,2020-01-02,P,AAPL,1,295.08,@ TI,N,NaN,0,1198,3,,40006429377792,NaN,0
4,4,2020-01-02 04:00:28.894835,2020-01-02,P,AAPL,3,295.10,@ TI,N,NaN,0,1205,4,,40028894459136,NaN,0
5,5,2020-01-02 04:00:30.021361,2020-01-02,P,AAPL,2,295.10,@ TI,N,NaN,0,1206,5,,40030020981248,NaN,0
6,6,2020-01-02 04:00:31.900055,2020-01-02,P,AAPL,7,295.10,@ TI,N,NaN,0,1208,6,,40031899679744,NaN,0
7,7,2020-01-02 04:00:33.047715,2020-01-02,P,AAPL,5,295.10,@ TI,N,NaN,0,1209,7,,40033047341056,NaN,0
8,8,2020-01-02 04:00:33.118294,2020-01-02,P,AAPL,5,295.10,@ TI,N,NaN,0,1210,8,,40033117919744,NaN,0
9,9,2020-01-02 04:00:33.118809,2020-01-02,P,AAPL,10,295.10,@ TI,N,NaN,0,1211,9,,40033118435584,NaN,0


In [6]:
quotes.head(10)

,Unnamed: 0,Time,Exchange,Symbol,Bid_Price,Bid_Size,Offer_Price,Offer_Size,Quote_Condition,Sequence_Number,...,Short_Sale_Restriction_Indicator,LULD_BBO_Indicator,SIP_Generated_Message_Identifier,NBBO_LULD_Indicator,Participant_Timestamp,FINRA_ADF_Timestamp,FINRA_ADF_Market_Participant_Quote_Indicator,Security_Status_Indicator,Date,YearMonth
0,0,2020-01-02 04:00:00.065165,P,AAPL,278.00,7.0,0.00,0.0,R,2228,...,0,NaN,NaN,NaN,40000064785664,NaN,NaN,,2020-01-02,202001
1,1,2020-01-02 04:00:00.065167,P,AAPL,278.00,14.0,0.00,0.0,R,2229,...,0,NaN,NaN,NaN,40000064787456,NaN,NaN,,2020-01-02,202001
2,2,2020-01-02 04:00:00.065170,P,AAPL,293.72,9.0,0.00,0.0,R,2230,...,0,NaN,NaN,NaN,40000064790784,NaN,NaN,,2020-01-02,202001
3,3,2020-01-02 04:00:00.065681,P,AAPL,293.72,9.0,327.56,1.0,R,2231,...,0,NaN,NaN,NaN,40000065302272,NaN,NaN,,2020-01-02,202001
4,4,2020-01-02 04:00:00.065738,P,AAPL,293.72,9.0,320.00,1.0,R,2232,...,0,NaN,NaN,NaN,40000065358592,NaN,NaN,,2020-01-02,202001
5,5,2020-01-02 04:00:00.065738,P,AAPL,293.72,9.0,310.00,1.0,R,2233,...,0,NaN,NaN,NaN,40000065360384,NaN,NaN,,2020-01-02,202001
6,6,2020-01-02 04:00:00.065744,P,AAPL,293.72,9.0,300.00,1.0,R,2235,...,0,NaN,NaN,NaN,40000065366528,NaN,NaN,,2020-01-02,202001
7,7,2020-01-02 04:00:00.065813,P,AAPL,293.72,9.0,299.97,5.0,R,2237,...,0,NaN,NaN,NaN,40000065433856,NaN,NaN,,2020-01-02,202001
8,8,2020-01-02 04:00:00.065816,P,AAPL,293.72,9.0,295.88,5.0,R,2238,...,0,NaN,NaN,NaN,40000065437440,NaN,NaN,,2020-01-02,202001
9,9,2020-01-02 04:00:00.068515,P,AAPL,295.00,1.0,295.88,5.0,R,2241,...,0,NaN,NaN,NaN,40000068136192,NaN,NaN,,2020-01-02,202001


In [7]:
pd.set_option('display.max_columns', None)

In [8]:
trades_columns = trades.columns
quotes_columns = quotes.columns

In [9]:
trades_columns

Index(['Unnamed: 0', 'Time', 'Date', 'Exchange', 'Symbol', 'Trade_Volume',
       'Trade_Price', 'Sale_Condition', 'Source_of_Trade',
       'Trade_Stop_Stock_Indicator', 'Trade_Correction_Indicator',
       'Sequence_Number', 'Trade_Id', 'Trade_Reporting_Facility',
       'Participant_Timestamp', 'Trade_Reporting_Facility_TRF_Timestamp',
       'Trade_Through_Exempt_Indicator'],
      dtype='object')

In [10]:
quotes_columns

Index(['Unnamed: 0', 'Time', 'Exchange', 'Symbol', 'Bid_Price', 'Bid_Size',
       'Offer_Price', 'Offer_Size', 'Quote_Condition', 'Sequence_Number',
       'National_BBO_Indicator', 'FINRA_BBO_Indicator',
       'FINRA_ADF_MPID_Indicator', 'Quote_Cancel_Correction',
       'Source_Of_Quote', 'Retail_Interest_Indicator',
       'Short_Sale_Restriction_Indicator', 'LULD_BBO_Indicator',
       'SIP_Generated_Message_Identifier', 'NBBO_LULD_Indicator',
       'Participant_Timestamp', 'FINRA_ADF_Timestamp',
       'FINRA_ADF_Market_Participant_Quote_Indicator',
       'Security_Status_Indicator', 'Date', 'YearMonth'],
      dtype='object')

In [72]:
all_events = pd.concat([trades, quotes], axis=0)

In [73]:
all_events.head(10)

,Unnamed: 0,Time,Date,Exchange,Symbol,Trade_Volume,Trade_Price,Sale_Condition,Source_of_Trade,Trade_Stop_Stock_Indicator,Trade_Correction_Indicator,Sequence_Number,Trade_Id,Trade_Reporting_Facility,Participant_Timestamp,Trade_Reporting_Facility_TRF_Timestamp,Trade_Through_Exempt_Indicator,Bid_Price,Bid_Size,Offer_Price,Offer_Size,Quote_Condition,National_BBO_Indicator,FINRA_BBO_Indicator,FINRA_ADF_MPID_Indicator,Quote_Cancel_Correction,Source_Of_Quote,Retail_Interest_Indicator,Short_Sale_Restriction_Indicator,LULD_BBO_Indicator,SIP_Generated_Message_Identifier,NBBO_LULD_Indicator,FINRA_ADF_Timestamp,FINRA_ADF_Market_Participant_Quote_Indicator,Security_Status_Indicator,YearMonth
0,0,2020-01-02 04:00:00.064010,2020-01-02,P,AAPL,3801.0,295.05,@ T,N,NaN,0.0,1185,1.0,,40000063617792,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2020-01-02 04:00:02.828485,2020-01-02,P,AAPL,1.0,295.08,@FTI,N,NaN,0.0,1195,2.0,,40002828108800,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2020-01-02 04:00:06.250392,2020-01-02,Q,AAPL,6.0,295.25,@ TI,N,NaN,0.0,1197,1.0,,40006250366823,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2020-01-02 04:00:06.429757,2020-01-02,P,AAPL,1.0,295.08,@ TI,N,NaN,0.0,1198,3.0,,40006429377792,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2020-01-02 04:00:28.894835,2020-01-02,P,AAPL,3.0,295.10,@ TI,N,NaN,0.0,1205,4.0,,40028894459136,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,2020-01-02 04:00:30.021361,2020-01-02,P,AAPL,2.0,295.10,@ TI,N,NaN,0.0,1206,5.0,,40030020981248,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,2020-01-02 04:00:31.900055,2020-01-02,P,AAPL,7.0,295.10,@ TI,N,NaN,0.0,1208,6.0,,40031899679744,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,2020-01-02 04:00:33.047715,2020-01-02,P,AAPL,5.0,295.10,@ TI,N,NaN,0.0,1209,7.0,,40033047341056,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,2020-01-02 04:00:33.118294,2020-01-02,P,AAPL,5.0,295.10,@ TI,N,NaN,0.0,1210,8.0,,40033117919744,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,2020-01-02 04:00:33.118809,2020-01-02,P,AAPL,10.0,295.10,@ TI,N,NaN,0.0,1211,9.0,,40033118435584,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
len(trades)+len(quotes)

2208691

# 2. Data Cleaning & Preprocessing

In [14]:
from sklearn.base import BaseEstimator, TransformerMixin

In [75]:
class PreprocessData(BaseEstimator, TransformerMixin):
    
    def __init__(self, dropped_after_hourse=True, droped_irregular_hours=True):
        self.dropped_after_hourse = dropped_after_hourse
        self.droped_irregular_hours = droped_irregular_hours
        
    
    def fit(self, X, y=None):
        return self
    
    def generate_mox_identifier(df: pd.DataFrame) -> pd.DataFrame:
        """Generate MOX Identifier"""
        # get participant timestamps
        participant_timestamps = df.index
        # convert timestamps to float
        fl_participant_timestamps = [float(ts.timestamp()*1000) for ts in participant_timestamps]
        # generate unique index for each timestamp
        time_mox_mapping = {ts: mox_idx for mox_idx, ts in enumerate(OrderedSet(fl_participant_timestamps))}
        # generate the mox_identifiers
        mox_identifiers = [time_mox_mapping[t] for t in fl_participant_timestamps]

        df['MOX_Identifiers'] = mox_identifiers

        return df
    
    def transform(self, X):
        
        X.drop(['Unnamed: 0', 'Time'], inplace=True, axis=1)
        
        # parse date and participant timestamp
        X['Date'] = pd.to_datetime(X['Date'])
        X['Participant_Timestamp'] = pd.to_datetime(
            X["Participant_Timestamp"].astype(str).str.zfill(15), format="%H%M%S%f"
        )
        
        # convert datetime to index
        X["Participant_Timestamp"] = X["Date"].apply(lambda x: x) + X["Participant_Timestamp"].apply(
            lambda x: timedelta(hours=x.hour, minutes=x.minute, seconds=x.second, microseconds=x.microsecond)
        )
        X.index = X["Participant_Timestamp"].values
        
        # remove rows of all NA
        X = X.dropna(axis=1, how="all")
        
        
        # sort data according to index
        X = X.sort_index()
        
        # drop after hours if specified
        if self.dropped_after_hourse:
            afterhours_idx = []
            for t in X.index:
                str_t = t.strftime("%H:%M:%S")
                if str_t < "09:00:00" or str_t > "16:00:00":
                    afterhours_idx.append(t)
            X.drop(afterhours_idx, inplace=True)
            
        # remove first and last 15 minutes of regular trading hours
        if self.droped_irregular_hours:
            irregular_idx = []
            for t in X.index:
                str_t = t.strftime("%H:%M:%S")
                if str_t < "09:45:00" or str_t > "15:45:00":
                    irregular_idx.append(t)
            X.drop(irregular_idx, inplace=True)
        
        
        X = generate_mox_identifier(X)
        
        return X

In [76]:
process_pipeline = make_pipeline(
    PreprocessData()
)

In [77]:
df_clean = process_pipeline.fit_transform(all_events)

In [78]:
df_clean.head(20)

,Date,Exchange,Symbol,Trade_Volume,Trade_Price,Sale_Condition,Source_of_Trade,Trade_Correction_Indicator,Sequence_Number,Trade_Id,Trade_Reporting_Facility,Participant_Timestamp,Trade_Reporting_Facility_TRF_Timestamp,Trade_Through_Exempt_Indicator,Bid_Price,Bid_Size,Offer_Price,Offer_Size,Quote_Condition,National_BBO_Indicator,Source_Of_Quote,Retail_Interest_Indicator,Short_Sale_Restriction_Indicator,SIP_Generated_Message_Identifier,NBBO_LULD_Indicator,Security_Status_Indicator,YearMonth,MOX_Identifiers
2020-01-02 09:45:00.001258,2020-01-02,N,AAPL,NaN,NaN,NaN,NaN,NaN,2025557,NaN,NaN,2020-01-02 09:45:00.001258,NaN,NaN,297.11,1.0,297.15,1.0,R,2.0,N,NaN,0.0,NaN,A,,202001.0,0
2020-01-02 09:45:00.001451,2020-01-02,N,AAPL,NaN,NaN,NaN,NaN,NaN,2025564,NaN,NaN,2020-01-02 09:45:00.001451,NaN,NaN,297.11,1.0,297.20,1.0,R,0.0,N,NaN,0.0,NaN,A,,202001.0,1
2020-01-02 09:45:00.001459,2020-01-02,N,AAPL,NaN,NaN,NaN,NaN,NaN,2025565,NaN,NaN,2020-01-02 09:45:00.001459,NaN,NaN,297.11,2.0,297.20,1.0,R,2.0,N,NaN,0.0,NaN,A,,202001.0,2
2020-01-02 09:45:00.001518,2020-01-02,N,AAPL,NaN,NaN,NaN,NaN,NaN,2025566,NaN,NaN,2020-01-02 09:45:00.001518,NaN,NaN,297.11,2.0,297.28,1.0,R,0.0,N,NaN,0.0,NaN,A,,202001.0,3
2020-01-02 09:45:00.001538,2020-01-02,N,AAPL,NaN,NaN,NaN,NaN,NaN,2025567,NaN,NaN,2020-01-02 09:45:00.001538,NaN,NaN,297.11,3.0,297.28,1.0,R,2.0,N,NaN,0.0,NaN,A,,202001.0,4
2020-01-02 09:45:00.012368,2020-01-02,Z,AAPL,NaN,NaN,NaN,NaN,NaN,2026234,NaN,NaN,2020-01-02 09:45:00.012368,NaN,NaN,297.08,2.0,297.13,2.0,R,2.0,N,NaN,0.0,NaN,A,,202001.0,5
2020-01-02 09:45:00.012486,2020-01-02,N,AAPL,NaN,NaN,NaN,NaN,NaN,2026243,NaN,NaN,2020-01-02 09:45:00.012486,NaN,NaN,297.11,2.0,297.28,1.0,R,2.0,N,NaN,0.0,NaN,A,,202001.0,6
2020-01-02 09:45:00.038245,2020-01-02,K,AAPL,NaN,NaN,NaN,NaN,NaN,2026679,NaN,NaN,2020-01-02 09:45:00.038245,NaN,NaN,297.04,1.0,297.13,1.0,R,0.0,N,NaN,0.0,NaN,A,,202001.0,7
2020-01-02 09:45:00.079739,2020-01-02,K,AAPL,NaN,NaN,NaN,NaN,NaN,2027059,NaN,NaN,2020-01-02 09:45:00.079739,NaN,NaN,297.04,1.0,297.13,2.0,R,0.0,N,NaN,0.0,NaN,A,,202001.0,8
2020-01-02 09:45:00.117069,2020-01-02,Q,AAPL,NaN,NaN,NaN,NaN,NaN,2027420,NaN,NaN,2020-01-02 09:45:00.117069,NaN,NaN,297.11,1.0,297.13,2.0,R,2.0,N,NaN,0.0,NaN,A,,202001.0,9


In [79]:
df_clean['Trade_Reporting_Facility'].unique()

array([nan, ' ', 'N', 'Q', 'B'], dtype=object)

## 2.1 Cleaning for Quotes and Trades using ColumnTransformer

In [20]:
from sortedcollections import OrderedSet
import time

In [21]:
trades_attributes = ['Unnamed: 0', 'Time', 'Date', 'Exchange', 'Symbol', 'Trade_Volume',
       'Trade_Price', 'Sale_Condition', 'Source_of_Trade',
       'Trade_Stop_Stock_Indicator', 'Trade_Correction_Indicator',
       'Sequence_Number', 'Trade_Id', 'Trade_Reporting_Facility',
       'Participant_Timestamp', 'Trade_Reporting_Facility_TRF_Timestamp',
       'Trade_Through_Exempt_Indicator']
quotes_attributes = ['Unnamed: 0', 'Time', 'Exchange', 'Symbol', 'Bid_Price', 'Bid_Size',
       'Offer_Price', 'Offer_Size', 'Quote_Condition', 'Sequence_Number',
       'National_BBO_Indicator', 'FINRA_BBO_Indicator',
       'FINRA_ADF_MPID_Indicator', 'Quote_Cancel_Correction',
       'Source_Of_Quote', 'Retail_Interest_Indicator',
       'Short_Sale_Restriction_Indicator', 'LULD_BBO_Indicator',
       'SIP_Generated_Message_Identifier', 'NBBO_LULD_Indicator',
       'Participant_Timestamp', 'FINRA_ADF_Timestamp',
       'FINRA_ADF_Market_Participant_Quote_Indicator',
       'Security_Status_Indicator', 'Date', 'YearMonth']

In [ ]:
#         # remove invalid trades price and volume
#         X.drop(X[X['Trade_Price'] < 0].index, inplace=True)
#         X.drop(X[X['Trade_Volume'] < 0].index, inplace=True)
        
#         # remove invalid quotes price
#         X.drop(X[X['Bid_Price'] < 0].index, inplace=True)
#         X.drop(X[X['Offer_Price'] < X['Bid_Price']].index, inplace=True)

In [22]:
df_test = df_clean.copy()

In [23]:
def generate_mox_identifier(df: pd.DataFrame) -> pd.DataFrame:
    """Generate MOX Identifier"""
    
    # get participant timestamps
    participant_timestamps = df.index
    # convert timestamps to float
    fl_participant_timestamps = [float(ts.timestamp()*1000) for ts in participant_timestamps]
    # generate unique index for each timestamp
    time_mox_mapping = {ts: mox_idx for mox_idx, ts in enumerate(OrderedSet(fl_participant_timestamps))}
    # generate the mox_identifiers
    mox_identifiers = [time_mox_mapping[t] for t in fl_participant_timestamps]
    
    df['MOX_Identifiers'] = mox_identifiers

    return df

In [24]:
now = time.time()
df_test = generate_mox_identifier(df_test)
print ("Generating MOX identifiers takes: {0:.3f}s".format(time.time()-now))

Generating MOX identifiers takes: 7.931s


In [25]:
df_test

,Date,Exchange,Symbol,Trade_Volume,Trade_Price,Sale_Condition,Source_of_Trade,Trade_Correction_Indicator,Sequence_Number,Trade_Id,Trade_Reporting_Facility,Participant_Timestamp,Trade_Reporting_Facility_TRF_Timestamp,Trade_Through_Exempt_Indicator,Bid_Price,Bid_Size,Offer_Price,Offer_Size,Quote_Condition,National_BBO_Indicator,Source_Of_Quote,Retail_Interest_Indicator,Short_Sale_Restriction_Indicator,SIP_Generated_Message_Identifier,NBBO_LULD_Indicator,Security_Status_Indicator,YearMonth,MOX_Identifiers
2020-01-02 04:00:00.063617,2020-01-02,P,AAPL,3801.0,295.05,@ T,N,0.0,1185,1.0,,2020-01-02 04:00:00.063617,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2020-01-02 04:00:00.065302,2020-01-02,P,AAPL,NaN,NaN,NaN,NaN,NaN,2231,NaN,NaN,2020-01-02 04:00:00.065302,NaN,NaN,293.72,9.0,327.56,1.0,R,4.0,N,NaN,0.0,NaN,NaN,,202001.0,1
2020-01-02 04:00:00.065358,2020-01-02,P,AAPL,NaN,NaN,NaN,NaN,NaN,2232,NaN,NaN,2020-01-02 04:00:00.065358,NaN,NaN,293.72,9.0,320.00,1.0,R,4.0,N,NaN,0.0,NaN,NaN,,202001.0,2
2020-01-02 04:00:00.065360,2020-01-02,P,AAPL,NaN,NaN,NaN,NaN,NaN,2233,NaN,NaN,2020-01-02 04:00:00.065360,NaN,NaN,293.72,9.0,310.00,1.0,R,4.0,N,NaN,0.0,NaN,NaN,,202001.0,3
2020-01-02 04:00:00.065366,2020-01-02,P,AAPL,NaN,NaN,NaN,NaN,NaN,2235,NaN,NaN,2020-01-02 04:00:00.065366,NaN,NaN,293.72,9.0,300.00,1.0,R,4.0,N,NaN,0.0,NaN,NaN,,202001.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-02 19:59:57.963903,2020-01-02,K,AAPL,70.0,301.21,@ TI,N,0.0,3133023,19386.0,,2020-01-02 19:59:57.963903,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019377
2020-01-02 19:59:57.963903,2020-01-02,K,AAPL,29.0,301.25,@ TI,N,0.0,3133024,19387.0,,2020-01-02 19:59:57.963903,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019377
2020-01-02 19:59:57.963903,2020-01-02,K,AAPL,NaN,NaN,NaN,NaN,NaN,30868600,NaN,NaN,2020-01-02 19:59:57.963903,NaN,NaN,301.10,46.0,301.25,1.0,R,2.0,N,NaN,0.0,NaN,NaN,,202001.0,2019377
2020-01-02 19:59:57.963903,2020-01-02,K,AAPL,NaN,NaN,NaN,NaN,NaN,30868599,NaN,NaN,2020-01-02 19:59:57.963903,NaN,NaN,301.10,46.0,301.25,2.0,R,0.0,N,NaN,0.0,NaN,NaN,,202001.0,2019377


In [57]:
t1 = df_test.index[30]
t1

Timestamp('2020-01-02 04:00:33.047341')